In [96]:
import alpha_vantage as av
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime
import requests
import json
import matplotlib.pyplot as plt
#training and predicting pipelines
from sklearn.metrics import fbeta_score, accuracy_score
#initial model evaluation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import time
#Random Forrest Training
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
import pickle 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV



In [97]:
def rate_today(df_rates):
    # Create a new dictionary with the date and other columns as the keys and values, respectively
    new_row = {'date': '2023-11-09',
            '5yr_rate': 4.65,
            '10yr_rate': 4.63,
            '30yr_rate': 4.77}

    # Pass a list of indices to the index argument of the pd.DataFrame() constructor
    new_row_df = pd.DataFrame(new_row, index=[0])

    # Convert the date column of the new_row_df DataFrame to type Timestamp
    new_row_df['date'] = pd.to_datetime(new_row_df['date'])

    # Convert the date column of the df_rates DataFrame to type Timestamp
    df_rates['date'] = pd.to_datetime(df_rates['date'])

    # Concatenate the new_row DataFrame with the df_rates DataFrame
    df_rates = pd.concat([df_rates, new_row_df], ignore_index=True)

    # Sort the df_rates DataFrame by the date column in descending order
    df_rates = df_rates.sort_values(by=['date'], ascending=False)

    return df_rates

def preprocess():
        api_key = 'ZWKEQ2PS4DJY66FW'
        #'ALUI1VJSESQR07TD'
        ts = TimeSeries(key=api_key)
        symbols=['QQQ', 'VXX']

        #date range for filtered_date
        start = '2009-01-30'
        end = datetime.now().strftime('%Y-%m-%d')

        df = pd.DataFrame()

        #pull in daily prices for each symbol
        for sym in symbols:
                data, meta_data = ts.get_daily(symbol=sym, outputsize='full')

                #filter data for the specified date range
                filtered_data = {date: values['4. close'] for date, values in data.items() if start <= date <= end}

                #convert filtered_data into a DF
                sym_df = pd.DataFrame.from_dict(filtered_data, orient='index')
                sym_df.index = pd.to_datetime(sym_df.index)

                #rename the column + use the symbol as the column header
                sym_df = sym_df.rename(columns={0: sym})

                #concatenate the DF for the current symbol to the main DF
                df = pd.concat([df, sym_df], axis=1)

        #calculate moving averages
        sma = [50, 100, 200]

        df_sma = df.sort_index(ascending=True) #make ascending to calc SMAs

        #at each sma value, calculate the corresponding sma for each ticker
        for num in sma:
                col_name = f'QQQ_SMA_{num}' #pass through column name for each sma
                df_sma[col_name] = df_sma['QQQ'].rolling(window=num).mean()

        sma_vxx = [2, 3]
        for num in sma_vxx:
                col_name = f'VXX_SMA_{num}' #pass through column name for each sma
                df_sma[col_name] = df_sma['VXX'].rolling(window=num).mean()

        df_sma = df_sma.sort_index(ascending=False)

        #remove rows with NaN
        df_sma_clean = df_sma.dropna()
        df_sma_clean = df_sma_clean.reset_index()
        df_sma_clean = df_sma_clean.rename(columns={'index': 'date'})
        df_sma_clean['date'] = pd.to_datetime(df_sma_clean['date'])

        url = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=5year&apikey='+api_key
        r = requests.get(url)
        data = r.json()

        # Create a list of DataFrames, one for each year
        df_5rate = []

        # Create a DataFrame for the year
        df_5rate = pd.DataFrame()
        for i in data['data']:
                df_5rate = pd.concat([df_5rate, pd.DataFrame({'date': i['date']}, index=[i['date']])], ignore_index=False)
                df_5rate.loc[i['date'], '5yr_rate'] = i['value']

        # Set the date column to a datetime type
        df_5rate['date'] = pd.to_datetime(df_5rate['date'])

        url = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey='+api_key
        r = requests.get(url)
        data = r.json()

        # Create a list of DataFrames, one for each year
        df_10rate = []

        # Create a DataFrame for the year
        df_10rate = pd.DataFrame()
        for i in data['data']:
                df_10rate = pd.concat([df_10rate, pd.DataFrame({'date': i['date']}, index=[i['date']])], ignore_index=False)
                df_10rate.loc[i['date'], '10yr_rate'] = i['value']

        # Set the date column to a datetime type
        df_10rate['date'] = pd.to_datetime(df_10rate['date'])

        url = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=30year&apikey='+api_key
        r = requests.get(url)
        data = r.json()

        # Create a list of DataFrames, one for each year
        df_30rate = []

        # Create a DataFrame for the year
        df_30rate = pd.DataFrame()
        for i in data['data']:
                df_30rate = pd.concat([df_30rate, pd.DataFrame({'date': i['date']}, index=[i['date']])], ignore_index=False)
                df_30rate.loc[i['date'], '30yr_rate'] = i['value']

        # Set the date column to a datetime type
        df_30rate['date'] = pd.to_datetime(df_30rate['date'])

        df_rates = df_5rate
        df_rates = df_rates.merge(df_10rate[['date', '10yr_rate']], on='date')
        df_rates = df_rates.merge(df_30rate[['date', '30yr_rate']], on='date')

        rates_backup = df_rates

        df_rates = rate_today(df_rates)

        symbol = 'QQQ' 
        #set up date column
        url = 'https://www.alphavantage.co/query?function=RSI&symbol='+symbol+'&interval=daily&time_period=10&series_type=open&apikey='+api_key
        rsi_resp = requests.get(url)
        rsi = json.loads(rsi_resp.content)

        rsi_data = rsi['Technical Analysis: RSI']

        df_rsi = pd.DataFrame([(date, rsi['RSI']) for date, rsi in rsi_data.items()], columns=['date', 'RSI'])
        df_rsi['date'] = pd.to_datetime(df_rsi['date'])

        #calculate moving averages
        rsi = [2, 3]

        df_rsi = df_rsi.sort_index(ascending=False) #make ascending to calc aves

        for num in rsi:
                col_name = f'RSI_{num}' #pass through column name for each sma
                df_rsi[col_name] = df_rsi['RSI'].rolling(window=num).mean()

        df_rsi = df_rsi.sort_index(ascending=True) #make ascending to calc aves

        df_continuous = df_sma_clean
        df_continuous = df_continuous.merge(df_rates[['date', '5yr_rate', '10yr_rate', '30yr_rate']], on='date')
        df_continuous = df_continuous.merge(df_rsi[['date', 'RSI', 'RSI_2', 'RSI_3']], on='date')

        return df_continuous

def traintest(days_predict, df_continuous):    
    
    df_cont = df_continuous.copy()
    df_cont = df_cont.drop('date', axis=1)
    df_cont = df_cont.apply(pd.to_numeric, errors='coerce')

    scaler = StandardScaler()
    df_continuous_scaled = scaler.fit_transform(df_cont)
    df_continuous_scaled = pd.DataFrame(df_continuous_scaled, columns=df_cont.columns)

    days_predict = days_predict

    # Define a function to add a new column to the DataFrame
    def add_qqq_greater_column(df):
        df['QQQ_'+str(days_predict)] = df['QQQ'] > df['QQQ'].shift(-days_predict)
        return df

    # Apply the function to the DataFrame
    df_continuous_scaled = add_qqq_greater_column(df_continuous_scaled.copy())

    df_continuous_scaled['QQQ_'+str(days_predict)] = df_continuous_scaled['QQQ_'+str(days_predict)].map({True: 1, False: 0})

    df_continuous_scaled = df_continuous_scaled.dropna()

    counts = df_continuous_scaled['QQQ_'+str(days_predict)].value_counts()
    pos = counts[1]
    neg = counts[0]

    resample_size = max(pos, neg)
    resample_size

    from sklearn.utils import resample

    positive = df_continuous_scaled[df_continuous_scaled['QQQ_'+str(days_predict)]==1]
    negative = df_continuous_scaled[df_continuous_scaled['QQQ_'+str(days_predict)]==0]

    if len(positive) != resample_size:
        positive_balanced = resample(positive, replace=True, n_samples=resample_size, random_state=1)
    else:
        positive_balanced = positive
    if len(negative) != resample_size:
        negative_balanced = resample(negative, replace=True, n_samples=resample_size, random_state=1)
    else:
        negative_balanced = negative

    df_continuous_balanced = pd.concat([positive_balanced, negative_balanced])

    price_change = df_continuous_balanced['QQQ_'+str(days_predict)]
    features = df_continuous_balanced[['VXX', 'VXX_SMA_2', 'VXX_SMA_3', 'QQQ_SMA_50', 'QQQ_SMA_100', 'QQQ_SMA_200', '5yr_rate', '10yr_rate', '30yr_rate', 'RSI', 'RSI_2', 'RSI_3']]

    # Split the 'features' and 'income' data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features,
                                                        price_change,
                                                        test_size = 0.2, 
                                                        random_state = 48)

    return X_train, X_test, y_train, y_test, df_continuous_balanced

def models(X_train, X_test, y_train, y_test):
    # Train a logistic regression model
    logistic_regression = LogisticRegression()
    logistic_regression.fit(X_train, y_train)

    # Train a support vector machine model
    svm = SVC()
    svm.fit(X_train, y_train)

    # Train a random forest classifier
    random_forest = RandomForestClassifier()
    random_forest.fit(X_train, y_train)

    # Train a gradient boosting machine model
    gbm = GradientBoostingClassifier()
    gbm.fit(X_train, y_train)

    # Evaluate the models on the test set
    logistic_regression_accuracy = logistic_regression.score(X_test, y_test)
    svm_accuracy = svm.score(X_test, y_test)
    random_forest_accuracy = random_forest.score(X_test, y_test)
    gbm_accuracy = gbm.score(X_test, y_test)

    # Calculate the F-score of each model
    logistic_regression_f1_score = f1_score(y_test, logistic_regression.predict(X_test), average='macro')
    svm_f1_score = f1_score(y_test, svm.predict(X_test), average='macro')
    random_forest_f1_score = f1_score(y_test, random_forest.predict(X_test), average='macro')
    gbm_f1_score = f1_score(y_test, gbm.predict(X_test), average='macro')

    # Print the accuracy of each model
    print('Logistic regression accuracy:', logistic_regression_accuracy.round(4))
    print('SVM accuracy:', svm_accuracy.round(4))
    print('Random forest accuracy:', random_forest_accuracy.round(4))
    print('GBM accuracy:', gbm_accuracy.round(4))

    # Print the F-score of each model
    print('Logistic regression F-score:', logistic_regression_f1_score.round(4))
    print('SVM F-score:', svm_f1_score.round(4))
    print('Random forest F-score:', random_forest_f1_score.round(4))
    print('GBM F-score:', gbm_f1_score.round(4))

    # Print the confusion matrix for the logistic regression model
    logistic_regression_confusion_matrix = confusion_matrix(y_test, logistic_regression.predict(X_test))
    print('Logistic regression confusion matrix:')
    print(logistic_regression_confusion_matrix)

    # Print the confusion matrix for the SVM model
    svm_confusion_matrix = confusion_matrix(y_test, svm.predict(X_test))
    print('SVM confusion matrix:')
    print(svm_confusion_matrix)

    # Print the confusion matrix for the random forest model
    random_forest_confusion_matrix = confusion_matrix(y_test, random_forest.predict(X_test))
    print('Random forest confusion matrix:')
    print(random_forest_confusion_matrix)

    # Print the confusion matrix for the GBM model
    gbm_confusion_matrix = confusion_matrix(y_test, gbm.predict(X_test))
    print('GBM confusion matrix:')
    print(gbm_confusion_matrix)

    """
    [[TP, FP],
    [FN, TN]]
    """

def optimize(X_train, X_test, y_train, y_test):

    # Define the hyperparameters to tune for the random forest model
    random_forest_hyperparameters = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 3, 5]
    }

    # Define the hyperparameters to tune for the GBM model
    gbm_hyperparameters = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.5]
    }

    # Create a grid search object for the random forest model
    random_forest_grid_search = GridSearchCV(RandomForestClassifier(), random_forest_hyperparameters, cv=5)

    # Create a grid search object for the GBM model
    gbm_grid_search = GridSearchCV(GradientBoostingClassifier(), gbm_hyperparameters, cv=5)

    # Fit the grid search objects to the training data
    random_forest_grid_search.fit(X_train, y_train)
    gbm_grid_search.fit(X_train, y_train)

    # Get the best random forest model
    best_random_forest_model = random_forest_grid_search.best_estimator_

    # Get the best GBM model
    best_gbm_model = gbm_grid_search.best_estimator_

    # Predict the classes of the test data using the best random forest model
    random_forest_y_pred = best_random_forest_model.predict(X_test)

    # Predict the classes of the test data using the best GBM model
    gbm_y_pred = best_gbm_model.predict(X_test)

    # Calculate the F-score for the random forest model
    random_forest_f1_score = f1_score(y_test, random_forest_y_pred, average='macro')

    # Calculate the confusion matrix for the random forest model
    random_forest_confusion_matrix = confusion_matrix(y_test, random_forest_y_pred)

    # Calculate the F-score for the GBM model
    gbm_f1_score = f1_score(y_test, gbm_y_pred, average='macro')

    # Calculate the confusion matrix for the GBM model
    gbm_confusion_matrix = confusion_matrix(y_test, gbm_y_pred)

    # Print the results
    print('Random forest F-score:', random_forest_f1_score)
    print('Random forest confusion matrix:')
    print(random_forest_confusion_matrix)

    print('GBM F-score:', gbm_f1_score)
    print('GBM confusion matrix:')
    print(gbm_confusion_matrix)

    return best_random_forest_model, best_gbm_model

def savemodels(best_random_forest_model, best_gbm_model):
    import pickle

    # Save the best random forest model
    with open('random_forest_model2.pkl', 'wb') as file_object:
        pickle.dump(best_random_forest_model, file_object)

    # Save the best GBM model
    with open('gbm_model2.pkl', 'wb') as file_object:
        pickle.dump(best_gbm_model, file_object)

    print('Models Saved')

def predict(model, df_continuous, days_predict):

    model = model

    with open(str(model)+'2.pkl', 'rb') as file_object:
        best_model = pickle.load(file_object)

    df_predicts = df_continuous[['VXX', 'VXX_SMA_2', 'VXX_SMA_3', 'QQQ_SMA_50', 'QQQ_SMA_100', 'QQQ_SMA_200', '5yr_rate', '10yr_rate', '30yr_rate', 'RSI', 'RSI_2', 'RSI_3']]

    # Assuming you have a dataframe named 'df_predicts'
    new_data_predictions = best_model.predict(df_predicts)
    new_data_probabilities = best_model.predict_proba(df_predicts)

    # Create a copy of the original dataframe 'df_discrete'
    predictions_df = df_continuous[['date', 'QQQ']]
    predictions_df = pd.DataFrame(predictions_df)

    # Add columns for predictions and probability estimates
    predictions_df['Predictions'] = new_data_predictions
    predictions_df['Probability_'+str(days_predict)+'_Day_Decline'] = new_data_probabilities[:, 0]
    predictions_df['Probability_'+str(days_predict)+'_Day_Rise'] = new_data_probabilities[:, 1]

    # Round the probabilities to the nearest thousandth
    predictions_df['Probability_'+str(days_predict)+'_Day_Decline'] = predictions_df['Probability_'+str(days_predict)+'_Day_Decline'].round(4)
    predictions_df['Probability_'+str(days_predict)+'_Day_Rise'] = predictions_df['Probability_'+str(days_predict)+'_Day_Rise'].round(4)
    
    '''
    idx_pred = days_predict - 1

    # Create a new column called "Prediction_n_Days_From_Now"
    predictions_df["Prediction_"+str(days_predict)+"_Days_Ago"] = predictions_df["Predictions"].shift(-idx_pred)
    predictions_df["QQQ_"+str(days_predict)+"_Days_Ago"] = predictions_df["QQQ"].shift(-idx_pred)

    # Create a new column called "QQQ_n_Days_Ago_Greater_Than_QQQ"
    predictions_df["Actual"] = predictions_df["QQQ_"+str(days_predict)+"_Days_Ago"] < predictions_df["QQQ"]
    # Convert the new column to integers
    predictions_df["Actual"] = predictions_df["Actual"].astype(int)

    # Create a new column called "QQQ_n_Days_Ago_Greater_Than_QQQ"
    predictions_df["Accuracy"] = predictions_df["Prediction_"+str(days_predict)+"_Days_Ago"] == predictions_df["Actual"]
    # Convert the new column to integers
    predictions_df["Accuracy"] = predictions_df["Accuracy"].astype(int)

    '''
    
    predictions = predictions_df

    predictions.to_csv(str(model)+'preds2.csv', index=False)

    return predictions

In [98]:
df_continuous = preprocess()
df_continuous.head(5)

,date,QQQ,VXX,QQQ_SMA_50,QQQ_SMA_100,QQQ_SMA_200,VXX_SMA_2,VXX_SMA_3,5yr_rate,10yr_rate,30yr_rate,RSI,RSI_2,RSI_3
0,2023-11-09,370.0700,20.8000,364.2944,367.5613,342.93875,20.400,20.393333,4.65,4.63,4.77,69.8027,69.50350,68.468267
1,2023-11-08,372.9400,20.0000,364.4302,367.5296,342.52705,20.190,20.286667,4.51,4.49,4.64,69.2043,67.80105,66.719467
2,2023-11-07,372.7000,20.3800,364.4668,367.4795,342.10420,20.430,20.680000,4.53,4.58,4.75,66.3978,65.47705,63.804233
3,2023-11-06,369.2100,20.4800,364.3480,367.4551,341.68550,20.830,21.116667,4.60,4.67,4.84,64.5563,62.50745,60.531933
4,2023-11-03,367.7100,21.1800,364.2442,367.4220,341.25285,21.435,21.776667,4.49,4.57,4.77,60.4586,58.51975,52.520433


In [99]:
def relative(df_continuous):
    df_continuous2 = df_continuous.copy()
    df_continuous2 = pd.DataFrame(df_continuous2)

    # Convert the specified columns to numeric
    columns_to_convert = [
        "QQQ",
        'VXX',
        "QQQ_SMA_50",
        "QQQ_SMA_100",
        "QQQ_SMA_200",
        "VXX_SMA_2",
        "VXX_SMA_3",
        "5yr_rate",
        "10yr_rate",
        "30yr_rate",
        "RSI",
        "RSI_2",
        "RSI_3"
    ]

    for col in columns_to_convert:
        df_continuous2[col] = pd.to_numeric(df_continuous2[col], errors='coerce')

    # Change to relative data
    df_continuous2 = df_continuous2.sort_index(ascending=False)

    # Calculate QQQSMA50/QQQ
    df_continuous2['QQQ_SMA_50'] = df_continuous2['QQQ_SMA_50'] / df_continuous2['QQQ']

    # Calculate QQQSMA100/QQQ
    df_continuous2['QQQ_SMA_100'] = df_continuous2['QQQ_SMA_100'] / df_continuous2['QQQ']

    # Calculate QQQSMA200/QQQ
    df_continuous2['QQQ_SMA_200'] = df_continuous2['QQQ_SMA_200'] / df_continuous2['QQQ']

    # Calculate VXX_SMA_2/VXX
    df_continuous2['VXX_SMA_2'] = df_continuous2['VXX_SMA_2'] / df_continuous2['VXX']

    # Calculate VXX_SMA_3/VXX
    df_continuous2['VXX_SMA_3'] = df_continuous2['VXX_SMA_3'] / df_continuous2['VXX']

    # Calculate the current value/the trailing 7 day average for 5yr_rate
    df_continuous2['5yr_rate_ave'] = df_continuous2['5yr_rate'].rolling(window=7).mean()
    df_continuous2['5yr_rate'] = df_continuous2['5yr_rate'] / df_continuous2['5yr_rate_ave']
    df_continuous2 = df_continuous2.drop('5yr_rate_ave', axis=1)

    # Calculate the current value/the trailing 7 day average for 10yr_rate
    df_continuous2['10yr_rate_ave'] = df_continuous2['10yr_rate'].rolling(window=7).mean()
    df_continuous2['10yr_rate'] = df_continuous2['10yr_rate'] / df_continuous2['10yr_rate_ave']
    df_continuous2 = df_continuous2.drop('10yr_rate_ave', axis=1)

    # Calculate the current value/the trailing 7 day average for 30yr_rate
    df_continuous2['30yr_rate_ave'] = df_continuous2['30yr_rate'].rolling(window=7).mean()
    df_continuous2['30yr_rate'] = df_continuous2['30yr_rate'] / df_continuous2['30yr_rate_ave']
    df_continuous2 = df_continuous2.drop('30yr_rate_ave', axis=1)

    # Calculate RSI_2/RSI
    df_continuous2['RSI_2'] = df_continuous2['RSI_2'] / df_continuous2['RSI']

    # Calculate RSI_3/RSI
    df_continuous2['RSI_3'] = df_continuous2['RSI_3'] / df_continuous2['RSI']

    df_continuous2 = df_continuous2.sort_index(ascending=True)
    df_continuous2 = df_continuous2.dropna()

    return df_continuous2

In [107]:
df_continuous = relative(df_continuous)

days_predict = 22

X_train, X_test, y_train, y_test, df_continuous_scaled = traintest(days_predict, df_continuous)

models(X_train, X_test, y_train, y_test)

Logistic regression accuracy: 0.8926
SVM accuracy: 0.9027
Random forest accuracy: 0.9463
GBM accuracy: 0.9262
Logistic regression F-score: 0.8926
SVM F-score: 0.9026
Random forest F-score: 0.9463
GBM F-score: 0.9261
Logistic regression confusion matrix:
[[398  42]
 [ 54 400]]
SVM confusion matrix:
[[413  27]
 [ 60 394]]
Random forest confusion matrix:
[[428  12]
 [ 36 418]]
GBM confusion matrix:
[[424  16]
 [ 50 404]]


In [120]:
# Create a copy of the original dataframe 'df_discrete'
predictions_df = pd.merge(df_continuous['date'], df_continuous_scaled, left_index=True, right_index=True)

# Drop the scaled QQQ column from the predictions_df DataFrame
predictions_df = predictions_df.drop('QQQ', axis=1)

# Add historic close price to output
predictions_df['QQQ'] = df_continuous['QQQ']
predictions_df = predictions_df.drop_duplicates()
#'gbm_model'
#'random_forest_model'
model = 'random_forest_model'

predictions = predict(model, predictions_df, days_predict)

predictions.head(10)

,date,QQQ,Predictions,Probability_22_Day_Decline,Probability_22_Day_Rise
0,2023-11-09,370.07,1,0.3505,0.6495
1,2023-11-08,372.94,1,0.0396,0.9604
2,2023-11-07,372.70,1,0.1175,0.8825
3,2023-11-06,369.21,1,0.0268,0.9732
4,2023-11-03,367.71,1,0.1319,0.8681
5,2023-11-02,363.44,1,0.2499,0.7501
6,2023-11-01,356.96,1,0.4011,0.5989
7,2023-10-31,350.87,1,0.4237,0.5763
8,2023-10-30,349.20,1,0.4301,0.5699
10,2023-10-26,343.66,0,0.9367,0.0633


In [51]:
best_random_forest_model, best_gbm_model = optimize(X_train, X_test, y_train, y_test)

savemodels(best_random_forest_model, best_gbm_model)

Random forest F-score: 0.874498716689173
Random forest confusion matrix:
[[398  42]
 [ 69 376]]
GBM F-score: 0.9265442936951316
GBM confusion matrix:
[[415  25]
 [ 40 405]]
